In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['load-data']

In [2]:
# Parameters
upstream = {"load-data": {"nb": "/Users/ellis/Documents/MSc/2. MLiS II/picar/executed-notebooks/load-data.ipynb", "images": "/Users/ellis/Documents/MSc/2. MLiS II/picar/products/img-loaded.npy", "labels": "/Users/ellis/Documents/MSc/2. MLiS II/picar/products/lab-loaded.npy"}}
product = {"nb": "/Users/ellis/Documents/MSc/2. MLiS II/picar/executed-notebooks/nvidia-model.ipynb", "model": "/Users/ellis/Documents/MSc/2. MLiS II/picar/products/model-nvidia/model"}


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
labels = np.load(upstream['load-data']['labels'])
images = np.load(upstream['load-data']['images'])

In [5]:
images.max()

1.0

In [6]:
log_dir = "products/model-nvidia/train_logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
inputs = keras.Input(shape=(40, 60, 3))
l = Conv2D(24, (5, 5), strides=(2, 2))(inputs)
l = Conv2D(36, (5, 5), strides=(1, 1))(l)
l = Conv2D(48, (5, 5), strides=(1, 1))(l)
l = Conv2D(64, (3, 3), strides=(1, 1))(l)
l = Conv2D(64, (3, 3), strides=(1, 1))(l)
l = Flatten()(l)
l = Dense(1164)(l)
l = Dense(50)(l)
l = Dense(2)(l)

model = keras.Model(inputs=inputs, outputs=l, name="NVidia_model")
model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.KLDivergence()]
)

model.summary()

Metal device set to: Apple M1
Model: "NVidia_model"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 input_1 (InputLayer)        [(None, 40, 60, 3)]       0         


 conv2d (Conv2D)             (None, 18, 28, 24)        1824      


...

Total params: 7,342,480


Trainable params: 7,342,480


Non-trainable params: 0


_________________________________________________________________


2022-03-22 15:34:25.575205: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-22 15:34:25.575423: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
x = model.fit(
    images,
    labels,
    epochs=2,
    batch_size=100,
    validation_split=0.2,
    callbacks=[tensorboard_callback]
)

Epoch 1/2


2022-03-22 15:34:25.841849: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2022-03-22 15:34:26.397472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/2 [==============>...............] - ETA: 1s - loss: 0.2008 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 1.2975

2/2 [==============================] - ETA: 0s - loss: 2813.1240 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.6996

...

2/2 [==============================] - 4s 2s/step - loss: 2813.1240 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 0.6996 - val_loss: 20950.2871 - val_accuracy: 0.0000e+00 - val_kullback_leibler_divergence: 22.0122


Epoch 2/2


1/2 [==============>...............] - ETA: 0s - loss: 20723.8672 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 21.6847

2/2 [==============================] - ETA: 0s - loss: 13114.0020 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 13.4439

2/2 [==============================] - 2s 2s/step - loss: 13114.0020 - accuracy: 0.0000e+00 - kullback_leibler_divergence: 13.4439 - val_loss: 109.6314 - val_accuracy: 0.0000e+00 - val_kullback_leibler_divergence: -0.2761


In [9]:
model.save("products/model-nvidia/model")

2022-03-22 15:34:31.398541: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: products/model-nvidia/model/assets
